In [1]:
import pandas as pd

In [2]:
def create_analysis_df(df, cols):
    analysis_results = []
    
    for col in cols:
        count = df[col].count()
        empty_count = df[col].isnull().sum()
        distinct_count = df[col].nunique()
        total = len(df[col])
        
        # Add data to list
        analysis_results.append({
            "Column": col,
            "Valid (%)": round((count - empty_count) / total * 100, 2),
            "Empty (%)": round(empty_count / total * 100, 2),
            "Distinct": f"{distinct_count}/{total}"
        })
        
    df_analysis = pd.DataFrame(analysis_results)
        
    return df_analysis

Preprocessing products.csv

In [3]:
df_product = pd.read_csv("../Data/products.csv")

df_product.dropna(how='all', inplace=True)

duplicate_products = df_product[df_product.duplicated(keep=False)]
df_product.drop_duplicates(inplace=True)
df_product = df_product.drop_duplicates(subset="Product ID", keep=False)

print("Duplicates removed!")
print("Some duplicated products:")
duplicate_products.head(5)

Duplicates removed!
Some duplicated products:


,Product ID,VN Name,EN Name,Brand,Category,Actual Price,Discount Price,Discount Rate,Variant,Star,Rating,Sold,Link,Image
5319,110972,[Mua 1 Tặng 1] Eucerin Tinh Chất Sáng Da + Tặn...,Pro ACNE Solution Triple Effect Serum 40ml + C...,Eucerin,Chăm Sóc Da Mặt / Bộ Chăm Sóc Da Mặt,649000,584000,10,2 món,0.0,0,NaN,https://hasaki.vn/san-pham/mua-1-tang-1-euceri...,https://media.hcdn.vn/catalog/product/p/r/prom...
5320,110972,[Mua 1 Tặng 1] Eucerin Tinh Chất Sáng Da + Tặn...,Pro ACNE Solution Triple Effect Serum 40ml + C...,Eucerin,Chăm Sóc Da Mặt / Bộ Chăm Sóc Da Mặt,649000,584000,10,2 món,0.0,0,NaN,https://hasaki.vn/san-pham/mua-1-tang-1-euceri...,https://media.hcdn.vn/catalog/product/p/r/prom...
5519,106971,"Kem Chống Nắng Nivea Hỗ Trợ Mờ Đốm Nâu, Ngừa T...",Luminous630 SpotClear Sun Protect SPF50 PA+++ ...,Nivea,Chăm Sóc Da Mặt / Chống Nắng Da Mặt,400000,392000,2,40ml,0.0,0,NaN,https://hasaki.vn/san-pham/kem-chong-nang-nive...,https://media.hcdn.vn/catalog/product/f/a/face...
5521,106971,"Kem Chống Nắng Nivea Hỗ Trợ Mờ Đốm Nâu, Ngừa T...",Luminous630 SpotClear Sun Protect SPF50 PA+++ ...,Nivea,Chăm Sóc Da Mặt / Chống Nắng Da Mặt,400000,392000,2,40ml,0.0,0,NaN,https://hasaki.vn/san-pham/kem-chong-nang-nive...,https://media.hcdn.vn/catalog/product/f/a/face...
5557,78574,Chì Kẻ Mày Australis Siêu Mảnh Sắc Nét Màu Nâu,Micro Brow Pencil - Brown,australis,Trang Điểm / Trang Điểm Mắt / Kẻ Mày,230000,184000,20,NaN,4.0,0,NaN,https://hasaki.vn/san-pham/chi-van-ke-may-sieu...,https://media.hcdn.vn/catalog/product/f/a/face...


Examine types of columns

In [4]:
product_cols = df_product.columns.tolist()

for col in product_cols:
    if df_product[col].dtype == "object":
        df_product[col] = df_product[col].astype("string")  # Convert to string type
    print(f"Type of '{col}':", df_product[col].dtype)

Type of 'Product ID': int64
Type of 'VN Name': string
Type of 'EN Name': string
Type of 'Brand': string
Type of 'Category': string
Type of 'Actual Price': int64
Type of 'Discount Price': int64
Type of 'Discount Rate': int64
Type of 'Variant': string
Type of 'Star': float64
Type of 'Rating': int64
Type of 'Sold': float64
Type of 'Link': string
Type of 'Image': string


In [5]:
df_analysis_product = create_analysis_df(df_product, product_cols)

print("Analysis of products.csv:")
print(df_analysis_product)

Analysis of products.csv:
            Column  Valid (%)  Empty (%)   Distinct
0       Product ID     100.00       0.00  6540/6540
1          VN Name     100.00       0.00  6539/6540
2          EN Name      87.37       6.31  4227/6540
3            Brand      99.97       0.02   438/6540
4         Category      92.35       3.82   131/6540
5     Actual Price     100.00       0.00   875/6540
6   Discount Price     100.00       0.00   817/6540
7    Discount Rate     100.00       0.00    82/6540
8          Variant      95.41       2.29  1081/6540
9             Star     100.00       0.00     6/6540
10          Rating     100.00       0.00    98/6540
11            Sold      50.06      24.97   538/6540
12            Link     100.00       0.00  6540/6540
13           Image     100.00       0.00  6538/6540


Assessment: 
- Fields containing empty values: EN Name, Brand, Category, Variant, Sold. These fields do not really need to be filled when the crawling could not get the data
The field 'Sold': Null values can be replaced by '0'

- Fully valid fields: Product ID, VN Name, Actual Price, Discount Price, Discount Rate, Star, Rating, Link, Image
- Fully distinct fields: Product ID, VN Name, Link
- Primary Key: Product ID

In [6]:
# Fill null "Sold" values with '0'
df_product["Sold"].fillna(0, inplace=True)

# Re-test the dropping function
df_analysis_product = create_analysis_df(df_product, product_cols)
print("Analysis of products.csv:")
print(df_analysis_product)

Analysis of products.csv:
            Column  Valid (%)  Empty (%)   Distinct
0       Product ID     100.00       0.00  6540/6540
1          VN Name     100.00       0.00  6539/6540
2          EN Name      87.37       6.31  4227/6540
3            Brand      99.97       0.02   438/6540
4         Category      92.35       3.82   131/6540
5     Actual Price     100.00       0.00   875/6540
6   Discount Price     100.00       0.00   817/6540
7    Discount Rate     100.00       0.00    82/6540
8          Variant      95.41       2.29  1081/6540
9             Star     100.00       0.00     6/6540
10          Rating     100.00       0.00    98/6540
11            Sold     100.00       0.00   539/6540
12            Link     100.00       0.00  6540/6540
13           Image     100.00       0.00  6538/6540


Save the processed products.csv

In [7]:
df_product.to_csv("../Data/processed_products.csv", index=False)

Preprocessing reviews.csv

In [10]:
df_review = pd.read_csv("../Data/reviews.csv")

df_review.dropna(how='all', inplace=True)

duplicate_reviews = df_review[df_review.duplicated(keep=False)]
df_review.drop_duplicates(inplace=True)

print("Duplicates removed!")
print("Some duplicated products:")
duplicate_reviews.head(5)

Duplicates removed!
Some duplicated products:


,Product ID,Username,Time,Stars,Review
6735,1641,phan thanh trúc,2024-12-05 22:59:00,5.0,t
6736,1641,phan thanh trúc,2024-12-05 22:59:00,5.0,t
20693,1640,phan thanh trúc,2024-12-05 22:59:00,5.0,t
20694,1640,phan thanh trúc,2024-12-05 22:59:00,5.0,t
30892,96705,Nguyenngocthanh,2023-08-12 12:23:00,5.0,NaN


In [11]:
review_cols = df_review.columns.tolist()

for col in review_cols:
    if df_review[col].dtype == "object":
        df_review[col] = df_review[col].astype("string")  # Convert to string type
    print(f"Type of '{col}':", df_review[col].dtype)

Type of 'Product ID': int64
Type of 'Username': string
Type of 'Time': string
Type of 'Stars': float64
Type of 'Review': string


In [12]:
df_analysis_review = create_analysis_df(df_review, review_cols)
print("Analysis of reviews.csv:")
print(df_analysis_review)

Analysis of reviews.csv:
       Column  Valid (%)  Empty (%)     Distinct
0  Product ID     100.00       0.00   3814/49066
1    Username      99.95       0.02   7866/49066
2        Time     100.00       0.00  15000/49066
3       Stars     100.00       0.00      2/49066
4      Review      93.12       3.44  13540/49066


Assessment:
- Fully valid fields: Product ID, Time, Stars
- Fields containing empty values: Username, Review. 
The field "Review" cannot be filled.
The field "Username" cannot be filled, should not be empty to identify the combined primary key
- Fully distinct values: None -> Primary Key: Combine [Product ID, Username, Time]
- 'Stars' has only 2 distinct values which can be predicted as 4.0 and 5.0 stars

Drop reviews not containing Username

In [13]:
df_review = df_review.dropna(subset=["Username"])

# Re-test the dropping function
df_analysis_review = create_analysis_df(df_review, review_cols)
print("Analysis of reviews.csv:")
print(df_analysis_review)

Analysis of reviews.csv:
       Column  Valid (%)  Empty (%)     Distinct
0  Product ID     100.00       0.00   3814/49054
1    Username     100.00       0.00   7866/49054
2        Time     100.00       0.00  14993/49054
3       Stars     100.00       0.00      2/49054
4      Review      93.11       3.44  13533/49054


Save the processed reviews.csv

In [14]:
df_review.to_csv("../Data/processed_reviews.csv", index=False)